In [1]:
import sys; sys.path.append('../')

In [2]:
from language_fragments import BuildSolver, get_config

In [3]:
config = get_config('solver')

In [4]:
solver = BuildSolver(config)

INFO:language_fragments.base.RuleParser:Loaded parser, with grammar=RuleGrammar, cache_size=3000, extra_reserve=30
INFO:language_fragments.base.SyllogisticSMTSolver:Created solver, cache_size=3000


**Main Rule Language** The current *rule_parser* and rule language  implemented the following language
fragments (based on work in
[this work](http://www.cs.man.ac.uk/~ipratt/papers/nat_lang/hst.pdf)):

| fragment 	| syntactic forms 	|
|-	|-	|
| **Basic syllogistic (BSyl)** 	| Every $cnoun$ {is, is not} a $cnoun$, Some $cnoun$ {is a, is not} $cnoun$, <noun> {is, is not} a $cnoun$, No $cnoun$ {is,is not} a $cnoun$ 	|
| **BSyl + transitive verbs (TV)** 	| {Some,Every, No} $cnoun$ (does not) $verb$ {Some,Every,No} $cnoun$, {Some,Every,No} $cnoun$ (does not) $verb$ $noun$, $noun$ (does not) $verb$ $noun$ 	|
| **BSyl + relative clauses (RC)** 	| {Some, Every, No} $cnoun$ who {is,is not} a $cnoun$ {is, is not} a $cnoun$  	|
| **BSyl + TV + RC** 	| {Some, Every, No} $cnoun$ who {is,is not} a $cnoun$ $verb$ {Some,Every,No} $cnoun$, {Some, Every, No} $cnoun$ who <verb> {Some,Every,No} $cnoun$ $verb$ {Some,Every,No} $cnoun$, 	|

where **$cnoun$**,**$verb$**,**$noun$** are place-holders that denote
*count nouns*, *verbs*, *nouns* (respectively)

In [5]:
solver.prove(["All Aa is a Bb"],debug=True)

ForAll([x],Implies(Aa(x),Bb(x)))


theory=sat, query=None

In [6]:
solver.prove(["All Aa is a Bb","No Cc is a Bb"],debug=True)

ForAll([x],Implies(Aa(x),Bb(x))) ForAll([x],Implies(Cc(x),Not(Bb(x))))


theory=sat, query=None

In [7]:
solver.prove(["All Aa is a Bb",
              "No Cc is a Bb",
              "Every Cc is a Bb",
              "John is a Cc"],debug=True)

ForAll([x],Implies(Aa(x),Bb(x))) ForAll([x],Implies(Cc(x),Not(Bb(x)))) ForAll([x],Implies(Cc(x),Bb(x))) Cc(john)


theory=unsat, query=None

In [8]:
solver.prove(["All Aa is a Bb",
              "All Aa is not a Bb"],debug=True)

ForAll([x],Implies(Aa(x),Bb(x))) ForAll([x],Implies(Aa(x),Not(Bb(x))))


theory=sat, query=None

In [9]:
solver.prove(["Every man is a mortal",
              "socrates is a man",
              "socrates is not a mortal"],debug=True)

ForAll([x],Implies(Man(x),Mortal(x))) Man(socrate) Not(Mortal(socrate))


theory=unsat, query=None

In [10]:
solver.prove(["Every man is a mortal","socrates is a man"],query="socrates is not a mortal",debug=True)

ForAll([x],Implies(Man(x),Mortal(x))) Man(socrate)


theory=sat, query=contradiction

In [11]:
solver.prove(["Every man is a mortal",
              "socrates is a man"],query="socrates is a mortal")

theory=sat, query=entailment

In [12]:
solver.prove(["Every man is a mortal",
              "socrates is a man",
              "Every dog is a saint"],
             query="socrates is a saint")

theory=sat, query=unknown

In [13]:
solver.prove(["Every artist who is not a beekeeper is a carpenter",
              "No beekeeper is a dentist",
              "No carpenter is a dentist",
              "John is an artist"],debug=True)

ForAll([x],Implies(And(Not(Beekeeper(x)),Artist(x)),Carpenter(x))) ForAll([x],Implies(Beekeeper(x),Not(Dentist(x)))) ForAll([x],Implies(Carpenter(x),Not(Dentist(x)))) Artist(john)


theory=sat, query=None

In [14]:
solver.prove(["Every artist who is not a beekeeper is a carpenter",
              "No beekeeper is a dentist",
              "No carpenter is a dentist",
              "John is an artist"],
             query="No artist is a dentist")

theory=sat, query=entailment

In [15]:
solver.prove(["Every artist who is not a beekeeper is a carpenter",
              "No beekeeper is a dentist",
              "No carpenter is a dentist",
              "John is an artist"],
             query="John is a dentist")

theory=sat, query=contradiction

In [16]:
solver.prove(["Every philosopher despises some cynic",
              "Every gentleman is a philosopher",
              "Every cynic is a man",
              "Every man is a human",
              "Socrates is a gentleman"])

theory=sat, query=None

In [17]:
solver.prove(["Every philosopher despises some cynic",
              "Every gentleman is a philosopher",
              "Every cynic is a man",
              "Every man is a human",
              "Socrates is a gentleman"],
             query="Some gentleman despises some human")

theory=sat, query=entailment

In [18]:
solver.prove(["Every philosopher who is not a stoic is a cynic",
              "Every stoic is a man",
              "Every cynic is a man"],
             query="Every philosopher is a man")

theory=sat, query=entailment

In [19]:
solver.prove(["Every philosopher who is not a stoic is a cynic",
              "Every stoic is a man",
              "Every cynic is a man"],
             query="Some philosopher is a man",debug=True)

ForAll([x],Implies(And(Not(Stoic(x)),Philosopher(x)),Cynic(x))) ForAll([x],Implies(Stoic(x),Man(x))) ForAll([x],Implies(Cynic(x),Man(x)))


theory=sat, query=unknown

In [20]:
solver.prove(["Every philosopher who is not a stoic is a cynic",
              "Every stoic is a man",
              "Every cynic is a man",
              "john is a philosopher"],
             query="Some philosopher is a man",debug=True)

ForAll([x],Implies(And(Not(Stoic(x)),Philosopher(x)),Cynic(x))) ForAll([x],Implies(Stoic(x),Man(x))) ForAll([x],Implies(Cynic(x),Man(x))) Philosopher(john)


theory=sat, query=entailment

In [21]:
solver.prove(["Some artist admires no beekeeper",
              "Every beekeeper admires some artist"],
             query="Some artist is not a beekeeper",debug=True)

Exists([x],And(Artist(x),ForAll([y],Implies(Beekeeper(y),Not(Admire(x,y)))))) ForAll([x],Implies(Beekeeper(x),Exists([y],And(Artist(y),Admire(x,y)))))


theory=sat, query=entailment

In [22]:
solver.prove(["Every stoic admires every cynic",
              "No cynic admires any stoic",
             ],
             query="No stoic is a cynic")

theory=sat, query=entailment

In [23]:
solver.prove(["Every p is a q","Every o is a p"],query="Every o is a q")

theory=sat, query=entailment

In [24]:
solver.prove(["Every p is a q","Every o is a p"],query="Some o is not a q",debug=True)

ForAll([x],Implies(P(x),Q(x))) ForAll([x],Implies(O(x),P(x)))


theory=sat, query=contradiction

In [25]:
solver.prove(["Every p is a q","Every o is a p"],query="Every o is not a q",debug=True)

ForAll([x],Implies(P(x),Q(x))) ForAll([x],Implies(O(x),P(x)))


theory=sat, query=unknown

In [26]:
solver.prove(["Every p is a q","Some o is a p"],query="Some o is a q")

theory=sat, query=entailment

In [27]:
solver.prove(["No p is a q","Some o is a p"],query="Some o is not a q")

theory=sat, query=entailment

In [28]:
solver.prove(["Some p is a q"],query="Some q is a p")

theory=sat, query=entailment

In [29]:
solver.prove(["No p is a q"],query="No q is a p")

theory=sat, query=entailment

In [30]:
solver.prove(["Every p is a q",
              "Every o is a p"],query="Every o is not a p",debug=True)

ForAll([x],Implies(P(x),Q(x))) ForAll([x],Implies(O(x),P(x)))


theory=sat, query=unknown

In [31]:
solver.prove(["Every doctor who is not a surgeon is a gardener",
              "Every gardener is a philosopher",
              "John is a doctor",
              "No philosopher is not a gardener"],query="John is a gardener")

theory=sat, query=unknown